# Install

In [ ]:
!pip install crewai
# !pip install -U duckduckgo-search
!pip install --upgrade --quiet  langchain-google-genai pillow
!pip install --upgrade --quiet  wikipedia

# Import

In [14]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain.tools import DuckDuckGoSearchRun (for duck duck search)
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# API

In [15]:
# api = Your KEY API


In [16]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose = True,
                             temperature = 0.6, # ความหลากหลายของคำตอบ
                             google_api_key=api)

In [ ]:
# search_tool = DuckDuckGoSearchRun() 
search_tool =  WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Agent

In [18]:
researcher = Agent (
    role="Master Ghost Hunter",
  goal='Find the scariest places in the world and share that experiences.',
  backstory="""The seasoned Master Ghost Hunter, born into a family with a 
  mystical heritage, has explored haunted sites worldwide, from ancient European castles to 
  abandoned North American asylums and remote Asian temples. Armed with occult knowledge and advanced equipment""",
  verbose=True, # output เพิ่มมากขึ้น
  allow_delegation=False, # อนุญาติให้ทำงานร่วมกับ Agent ตัวอื่น
  llm=llm,
    tools=[search_tool],
)


writer = Agent(
  role='Senior Writer content',
  goal = "Expertly writes stories about haunted and frightening places.",
  backstory="""Meet the mysterious blogger who passionately writes eerie tales on her blog. 
  Born with a fascination for the macabre, she draws inspiration from haunted places and folklore, 
  captivating readers with chilling stories that leave them unnerved.""",
  verbose=True,
  allow_delegation=True,
  llm=llm
)

# Task

In [19]:
# Create tasks for your agents
task1 = Task(
  description="""Find and tell the 10 most haunted places from around the world that you think are the scariest in 2023. 
  The story must include the name and the place as well as the story that made the place scary in detail.""",
  agent=researcher
)

task2 = Task(
  description="""Write a scary story from the information you received to make it even scarier but not distort the truth, 
  as well as make it more interesting. The final answer has to be the 10 scariest places written to attract people to read..""",
  agent=writer
)

# Crew

In [20]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
  process = Process.sequential
)

In [21]:
# Get your crew to work!
result = crew.kickoff()



[DEBUG]: Working Agent: Master Ghost Hunter

[INFO]: Starting Task: Find and tell the 10 most haunted places from around the world that you think are the scariest in 2023. 
  The story must include the name and the place as well as the story that made the place scary in detail.


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Wikipedia
Action Input: Scariest places in the worldPage: Scariest Places on Earth
Summary: Scariest Places on Earth is an American paranormal reality television series that originally aired from October 23, 2000, to October 29, 2006, on Fox Family, and later ABC Family. The show was hosted by Linda Blair, with narration by Zelda Rubinstein. The show featured reported cases of the paranormal by detailing the location's history, and then sending an ordinary family to visit the location in a reality TV-style vigil.

Page: Houska Castle
Summary: Houska Castle is an early Gothic castle in the municipality of Blatce in the Libe

Gemini produced an empty response.
Unrecognized role: . Treating as a ChatMessage.


Do I need to use a tool? No
Final Answer: 1. **Houska Castle, Czech Republic:** Built atop a bottomless pit said to be a gateway to Hell, Houska Castle is believed to be haunted by demons trapped beneath its foundation. Visitors often report hearing strange noises and feeling a sense of unease while exploring the castle.

2. **The Winchester Mystery House, San Jose, California:** This bizarre Victorian mansion was built by Sarah Winchester, the widow of William Wirt Winchester, the heir to the Winchester rifle fortune. Sarah believed she was haunted by the ghosts of those killed by Winchester rifles, and she spent the rest of her life constructing the house in an attempt to appease them. The house is full of strange architectural features, such as staircases that lead nowhere and doors that open into walls.

3. **The Tower of London, London, England:** This historic fortress has been the site of many executions and imprisonments over the centuries. The most famous ghost that is said to

In [ ]:
print(result)